In [1]:
import time
import requests
import telebot
import datetime
import numpy as np
import pandas as pd
from IPython.display import clear_output
from binance import Binance
import datetime
# from pyti import money_flow_index
from pyti.money_flow import money_flow
from decimal import Decimal, ROUND_HALF_EVEN, ROUND_DOWN
import re


telega=False
sshort=False
order_cena=50
limit_balance=2200
persent_perezakup=0.03
bot = Binance(
    API_KEY=,
    API_SECRET=
)

In [2]:
# binance_check=0
def EMA(df, base, target, period, alpha=False):
    """
    Function to compute Exponential Moving Average (EMA)
    
    Args :
        df : Pandas DataFrame which contains ['date', 'open', 'high', 'low', 'close', 'volume'] columns
        base : String indicating the column name from which the EMA needs to be computed from
        target : String indicates the column name to which the computed data needs to be stored
        period : Integer indicates the period of computation in terms of number of candles
        alpha : Boolean if True indicates to use the formula for computing EMA using alpha (default is False)
        
    Returns :
        df : Pandas DataFrame with new column added with name 'target'
    """

    con = pd.concat([df[:period][base].rolling(window=period).mean(), df[period:][base]])
    
    if (alpha == True):
        # (1 - alpha) * previous_val + alpha * current_val where alpha = 1 / period
        df[target] = con.ewm(alpha=1 / period, adjust=False).mean()
    else:
        # ((current_val - previous_val) * coeff) + previous_val where coeff = 2 / (period + 1)
        df[target] = con.ewm(span=period, adjust=False).mean()
    
    df[target].fillna(0, inplace=True)
    return df

def ATR(df, period, ohlc=['Open', 'High', 'Low', 'Close']):
    """
    Function to compute Average True Range (ATR)
    
    Args :
        df : Pandas DataFrame which contains ['date', 'open', 'high', 'low', 'close', 'volume'] columns
        period : Integer indicates the period of computation in terms of number of candles
        ohlc: List defining OHLC Column names (default ['Open', 'High', 'Low', 'Close'])
        
    Returns :
        df : Pandas DataFrame with new columns added for 
            True Range (TR)
            ATR (ATR_$period)
    """
    atr = 'ATR_' + str(period)

    # Compute true range only if it is not computed and stored earlier in the df
    if not 'TR' in df.columns:
        df['h-l'] = df[ohlc[1]] - df[ohlc[2]]
        df['h-yc'] = abs(df[ohlc[1]] - df[ohlc[3]].shift())
        df['l-yc'] = abs(df[ohlc[2]] - df[ohlc[3]].shift())
         
        df['TR'] = df[['h-l', 'h-yc', 'l-yc']].max(axis=1)
         
        df.drop(['h-l', 'h-yc', 'l-yc'], inplace=True, axis=1)

    # Compute EMA of true range using ATR formula after ignoring first row
    EMA(df, 'TR', atr, period, alpha=True)
    
    return df

def SuperTrend(df, period, multiplier, ohlc=['Open', 'High', 'Low', 'Close']):
    """
    Function to compute SuperTrend
    
    Args :
        df : Pandas DataFrame which contains ['date', 'open', 'high', 'low', 'close', 'volume'] columns
        period : Integer indicates the period of computation in terms of number of candles
        multiplier : Integer indicates value to multiply the ATR
        ohlc: List defining OHLC Column names (default ['Open', 'High', 'Low', 'Close'])
        
    Returns :
        df : Pandas DataFrame with new columns added for 
            True Range (TR), ATR (ATR_$period)
            SuperTrend (ST_$period_$multiplier)
            SuperTrend Direction (STX_$period_$multiplier)
    """

    ATR(df, period, ohlc=ohlc)
    atr = 'ATR_' + str(period)
    st = 'ST_' + str(period) + '_' + str(multiplier)
    stx = 'STX_' + str(period) + '_' + str(multiplier)
    
    """
    SuperTrend Algorithm :
    
        BASIC UPPERBAND = (HIGH + LOW) / 2 + Multiplier * ATR
        BASIC LOWERBAND = (HIGH + LOW) / 2 - Multiplier * ATR
        
        FINAL UPPERBAND = IF( (Current BASICUPPERBAND < Previous FINAL UPPERBAND) or (Previous Close > Previous FINAL UPPERBAND))
                            THEN (Current BASIC UPPERBAND) ELSE Previous FINALUPPERBAND)
        FINAL LOWERBAND = IF( (Current BASIC LOWERBAND > Previous FINAL LOWERBAND) or (Previous Close < Previous FINAL LOWERBAND)) 
                            THEN (Current BASIC LOWERBAND) ELSE Previous FINAL LOWERBAND)
        
        SUPERTREND = IF((Previous SUPERTREND = Previous FINAL UPPERBAND) and (Current Close <= Current FINAL UPPERBAND)) THEN
                        Current FINAL UPPERBAND
                    ELSE
                        IF((Previous SUPERTREND = Previous FINAL UPPERBAND) and (Current Close > Current FINAL UPPERBAND)) THEN
                            Current FINAL LOWERBAND
                        ELSE
                            IF((Previous SUPERTREND = Previous FINAL LOWERBAND) and (Current Close >= Current FINAL LOWERBAND)) THEN
                                Current FINAL LOWERBAND
                            ELSE
                                IF((Previous SUPERTREND = Previous FINAL LOWERBAND) and (Current Close < Current FINAL LOWERBAND)) THEN
                                    Current FINAL UPPERBAND
    """
    
    # Compute basic upper and lower bands
    df['basic_ub'] = (df[ohlc[1]] + df[ohlc[2]]) / 2 + multiplier * df[atr]
    df['basic_lb'] = (df[ohlc[1]] + df[ohlc[2]]) / 2 - multiplier * df[atr]

    # Compute final upper and lower bands
    df['final_ub'] = 0.00
    df['final_lb'] = 0.00
    for i in range(period, len(df)):
        df['final_ub'].iat[i] = df['basic_ub'].iat[i] if df['basic_ub'].iat[i] < df['final_ub'].iat[i - 1] or df[ohlc[3]].iat[i - 1] > df['final_ub'].iat[i - 1] else df['final_ub'].iat[i - 1]
        df['final_lb'].iat[i] = df['basic_lb'].iat[i] if df['basic_lb'].iat[i] > df['final_lb'].iat[i - 1] or df[ohlc[3]].iat[i - 1] < df['final_lb'].iat[i - 1] else df['final_lb'].iat[i - 1]
       
    # Set the Supertrend value
    df[st] = 0.00
    for i in range(period, len(df)):
        df[st].iat[i] = df['final_ub'].iat[i] if df[st].iat[i - 1] == df['final_ub'].iat[i - 1] and df[ohlc[3]].iat[i] <= df['final_ub'].iat[i] else \
                        df['final_lb'].iat[i] if df[st].iat[i - 1] == df['final_ub'].iat[i - 1] and df[ohlc[3]].iat[i] >  df['final_ub'].iat[i] else \
                        df['final_lb'].iat[i] if df[st].iat[i - 1] == df['final_lb'].iat[i - 1] and df[ohlc[3]].iat[i] >= df['final_lb'].iat[i] else \
                        df['final_ub'].iat[i] if df[st].iat[i - 1] == df['final_lb'].iat[i - 1] and df[ohlc[3]].iat[i] <  df['final_lb'].iat[i] else 0.00 
                 
    # Mark the trend direction up/down
    df[stx] = np.where((df[st] > 0.00), np.where((df[ohlc[3]] < df[st]), 'down',  'up'), np.NaN)

    # Remove basic and final bands from the columns
    df.drop(['basic_ub', 'basic_lb', 'final_ub', 'final_lb'], inplace=True, axis=1)
    
    df.fillna(0, inplace=True)

    return(list(df['STX_'+ str(period) + '_' + str(multiplier)][97:100]))

def f(s):
    ls = s.split('.', 1)
    if len(ls) == 2 and re.match(r'\d*$', ls[1]):
        return len(ls[1].rstrip('0'))
    return 0

boot = telebot.TeleBot('950940715:AAFy0d1stHb45OhKoufE607oHm9zzCnwSrc')

def anal5(pre,niz_gran=20,up_gran=80):
        try:
            pre = pre.to_list()
        except:
            pre=pre
        if pre[2]<=niz_gran and pre[3]<=niz_gran and pre[4]>=niz_gran:
            return("Пробил нижнюю границу снизу вверх")
        elif pre[3]>=up_gran and pre[4]<=up_gran:
            return( "Пробил верхнюю границу сверху вниз")
        elif pre[0]>=pre[1]>=pre[2]>=pre[3]>=pre[4]:
            return( "5x падение")
        elif pre[0]<=pre[1]<=pre[2]<=pre[3]<=pre[4]:
            return( "5x рост")
        elif pre[0]>=pre[1]>=pre[2]>=pre[3]<=pre[4]:
            return("4x падение и рост")
        elif pre[0]<=pre[1]<=pre[2]<=pre[3]>=pre[4]:
            return("4x рост и падение")
        else:
            return("скучно")
        

def rma(x, n, y0):
        a = (n-1) / n
        ak = a**np.arange(len(x)-1, -1, -1)
        return np.r_[np.full(n, np.nan), y0, np.cumsum(ak * x) / ak / n + y0 * a**np.arange(1, len(x)+1)]
def rsi_of_symbol(klins):
    
    means=[]
    for klin in klins[1]:
        means.append(float(klin[4]))
    df =  pd.DataFrame({'close':means})    
    n = 14

    df['change'] = df['close'].diff()
    df['gain'] = df.change.mask(df.change < 0, 0.0)
    df['loss'] = -df.change.mask(df.change > 0, -0.0)
    df['avg_gain'] = rma(df.gain[n+1:].to_numpy(), n, np.nansum(df.gain.to_numpy()[:n+1])/n)
    df['avg_loss'] = rma(df.loss[n+1:].to_numpy(), n, np.nansum(df.loss.to_numpy()[:n+1])/n)
    df['rs'] = df.avg_gain / df.avg_loss
    df['rsi_14'] = 100 - (100 / (1 + df.rs))
    
    rsi=df['rsi_14']
    pre=rsi.loc[95:99]
    status=anal5(pre,23,81)
    pre = pre.to_list() 
    return(pre[4],status)
        
def macd_of_symbol(klins):
  
    means=[]
    times=[]
    for klin in klins[1]:
        means.append(float(klin[4]))
        times.append(datetime.datetime.fromtimestamp(klin[0]/1000.0) )
    df =  pd.DataFrame((zip(means,times)),columns =['Курс', 'время'])    
    exp1 = df["Курс"].ewm  (span=12, adjust=False).mean  ()
    exp2 = df["Курс"].ewm  (span=26, adjust=False).mean  ()
    macd = exp1-exp2
    exp3 = macd.ewm  (span=9, adjust=False).mean  ()
    hism_macd = macd-exp3
    
    preee=hism_macd.loc[0:99]
    preee = preee.to_list()
    mean_macd=hism_macd.loc[95:99].mean()# среднее гистограмки
    
 
    blue=macd.loc[95:99].mean()# синенькая линия
    pre1=macd.loc[95:99]
    pre1 = pre1.to_list() 
    orange=exp3.loc[95:99].mean() #оранжевая
    pre2=exp3.loc[95:99]
    pre2 = pre2.to_list() 
    pre=hism_macd.loc[95:99]
    status=anal5(pre,0,0)
    pre = pre.to_list() 
    
    return(pre[4],status,pre1[4],pre2[4])   
        
def mfi_of_symbol(klins):
    
    end,mmax,mmin,vol=[],[],[],[]
    
    for klin in klins[1]:
        end.append(float(klin[4]))
        mmax.append(float(klin[2]))
        mmin.append(float(klin[3]))
        vol.append(float(klin[5]))
        

    df = pd.DataFrame((zip(end,mmax,mmin,vol)),columns =['Close', 'High','Low','Volume'])
    
    typical_price = (df['Close'] + df['High'] + df['Low']) / 3
    period =  14
    money_flow = typical_price * df['Volume']
    positive_flow =[] #Create a empty list called positive flow
    negative_flow = [] #Create a empty list called negative flow
    #Loop through the typical price 
    for i in range(1, len(typical_price)):
        if typical_price[i] > typical_price[i-1]:#if the present typical price is greater than yesterdays typical price
            positive_flow.append(money_flow[i])# Then append money flow at position i-1 to the positive flow list
            negative_flow.append(0) #Append 0 to the negative flow list
        elif typical_price[i] < typical_price[i-1]:#if the present typical price is less than yesterdays typical price
            negative_flow.append(money_flow[i])# Then append money flow at position i-1 to negative flow list
            positive_flow.append(0)#Append 0 to the positive flow list
        else: #Append 0 if the present typical price is equal to yesterdays typical price
            positive_flow.append(0)
            negative_flow.append(0)
    #Get all of the positive and negative money flows within the time period

    positive_mf =[]
    negative_mf = [] 
    #Get all of the positive money flows within the time period
    for i in range(period-1, len(positive_flow)):
        positive_mf.append(sum(positive_flow[i+1-period : i]))
    #Get all of the negative money flows within the time period  
    for i in range(period-1, len(negative_flow)):
        negative_mf.append(sum(negative_flow[i+1-period : i]))
    
    
    mfi = 100 * (np.array(positive_mf) / (np.array(positive_mf)  + np.array(negative_mf) ))
    
    pre=mfi[len(mfi)-5:len(mfi)]
    status=anal5(pre,15,94)

    return(pre[4],status,pre)

def status_S_B_C(S,B,C):
    if S>0 and  B>0 and  C>0.5:
        return("status1")
    elif S>0 and B>0 and  C<-0.5:
        return("status2")
    elif S>0 and  B<0 and C>0.5:
        return("status3")
    elif S>0 and B<0 and C<-0.5:
        return("status4")
    elif S<0 and B>0 and C>0.5:
        return("status5")
    elif S<0 and B>0 and C<-0.5:
        return("status6")
    elif S<0 and B<0 and C>0.5:
        return("status7")
    elif S<0 and B<0 and C<-0.5:
        return("status8")
    else:
        return("status0")

def proverOCHKA(my_symbol,interval,L_rsi,L_mfi):
    klins=('klines', bot.klines(
    symbol=my_symbol,
    interval=interval,
    limit=100
    ))
    macd_v = macd_of_symbol(klins)[0] 
    line1_v = macd_of_symbol(klins)[2] 
    line2_v = macd_of_symbol(klins)[3] 
    rsi_v = rsi_of_symbol(klins)[0] 
    mfi_v = mfi_of_symbol(klins)[0]
    return(rsi_v<L_rsi and mfi_v<L_mfi and macd_v<0 and line1_v<0 and line2_v<0)


def prodavator(OLD_list_of_delta,OLD_list_podstrax):
#     time.sleep(5)
    orders=('openOrders', bot.openOrders())#bbbot \2
#     binance_check+=5
    sells,bays = [],[]
    for i in orders[1]:
        if i['side']=='SELL'and i['clientOrderId'][0:5]=='botic':
            sells.append(i)
        elif i['side']=='BUY'and i['clientOrderId'][0:5]=='botic':
            bays.append(i)
    NEW_list_of_delta = {}
    NEW_list_of_podstrax = {}
    list_of_id={}
    list_of_q={}
    print("\n Сейчас мы в перезакупке\n")
    list_of_symbols_sell=[]
    for sell in sells:
#         print(now_cena)
        b=('ticker/price', bot.tickerPrice(symbol=sell['symbol']))# bbbot 1
#         binance_check+=1
        now_cena=float(b[1]['price'])
        
        old_cena = float(sell['clientOrderId'].split('_')[1])/10000
        order=info(sell)
#         print("ТЕКУЩАЯ" ,now_cena, "СТАРАЯ",old_cena*(1-persent_perezakup),"ДЕЛЬТА",now_cena-old_cena*(1-persent_perezakup))
        
       
            
        if now_cena<old_cena*0.995:
            if float(order[3])>order_cena*1.5:
                if now_cena<old_cena*(1-persent_perezakup):
                    pokypator(order[0],order_cena,limit_balance,'1m',OLD_list_podstrax)
            else:
                gotovo=False
                if proverOCHKA(order[0],"15m",30,15):
                    if proverOCHKA(order[0],"1m",30,15):
                        if order[6]>=3:
                            pokypator(order[0],order_cena,limit_balance,'1m',OLD_list_podstrax)
                            gotovo=True
                if gotovo==False:
                    if order[6]>=6:
                        pokypator(order[0],order_cena,limit_balance,'1m',OLD_list_podstrax)


        list_of_id.update({order[0]:order[1]})
        list_of_q.update({order[0]:order[4]})
        NEW_list_of_delta.update({order[0]:order[6]})
        if float(order[3])>order_cena*1.5: # тоесть уже один раз купили и 1 раз перекупили
            NEW_list_of_podstrax.update({order[0]:True })
        else:
            NEW_list_of_podstrax.update({order[0]:False })
        
            
#     print(NEW_list_of_delta)        
    for old_delta in OLD_list_of_delta:
        for new_delta in NEW_list_of_delta:
            if old_delta == new_delta:
                if OLD_list_of_delta[old_delta] < 0.1:
                    if NEW_list_of_delta[new_delta] >= 0.1:
                        print('cancelOrder', bot.cancelOrder(
                        orderId=list_of_id[new_delta]  ,
                        symbol=new_delta,))
                        time.sleep(2)
                        aaa=bot.createOrder(
                        symbol=new_delta,
                        recvWindow=5000,
                        side='sell',
                        type='MARKET',
                        quantity=list_of_q[new_delta]
                        )
                        if telega:
                            textik='https://api.telegram.org/bot950940715:AAFy0d1stHb45OhKoufE607oHm9zzCnwSrc/sendMessage?chat_id=-1001360890249&text='
                            rrr= requests.get(textik+ str(old_delta)+ " сейчас продал по_маркету_01" )
    OLD_list_of_delta = NEW_list_of_delta
    OLD_list_podstrax = NEW_list_of_podstrax
#     print(OLD_list_podstrax)
    
    return(OLD_list_of_delta,OLD_list_podstrax)                    
                        
                        
def byatia(period,buys_symbol):
    klins=('klines', bot.klines(
        symbol=buys_symbol,
        interval=period,
        limit=100
    ))
#     binance_check+=1
    mini=[]
    maxi=[]
    for klin in klins[1]:
        mini.append(float(klin[3])) # Собрал минимумы по 100 свечам
        maxi.append(float(klin[2])) # Собрал максимумы по 100 свечам
    mmmeans=[]
    for i,j in zip(mini,maxi):
        mmmeans.append((1-float(i)/float(j))) # поделил минимум на максимум отнял это число от 1  
        # тем самым узнал на сколько % изменилась цена за свечу
    number = Decimal(sum(mmmeans) / len(mmmeans)) # нашел среднюю изменений тем самым узнал на сколько в среднем меняется цена
    end_nacenka=(number.quantize(Decimal("1."+str('0'*4)), ROUND_HALF_EVEN))# округлил полученное изменение цены до 4 знаков вниз
    return(float(end_nacenka)+0.0035) # учитывая 35 сотых процента коммисии возвращаю результат

def pokypator(buys_symbol,fiftin_USDT,ogranich,period,OLD_list_podstrax):
    time.sleep(3)
    print(OLD_list_podstrax)
    test_podstrax = 0
    if buys_symbol in OLD_list_podstrax:
        if OLD_list_podstrax[buys_symbol]:
            print("вот тут мы на зарезервированной монетке ", buys_symbol )
            test_podstrax = 0
        else:
            print("вот тут  мы перекупаем  не зарезервированную монетку ", buys_symbol )
            for i in OLD_list_podstrax:
                if (OLD_list_podstrax[i]):
                    test_podstrax+=fiftin_USDT
    else:
        print("вот тут мы на не зарезервированной монетке которую мы ещё не перезакупали ", buys_symbol )
        for i in OLD_list_podstrax:
            if (OLD_list_podstrax[i]):
                test_podstrax+=fiftin_USDT                
    print(" вот какое значение у новой переменной ",test_podstrax)
    b=('account', bot.account())# bbbot 5
#     binance_check+=5
    coins=b[1]['balances']
    for coin in coins:
        if coin['asset'] == 'USDT':
            total_USDT=float(coin['free'])# 1 узнали сколько совободных бабок
            break
    
    print(222222222222222222222222222222)
    print(total_USDT)
    print(222222222222222222222222222222)
    if (total_USDT)>=fiftin_USDT+2:# если бабок хватает на ордер

        orders=('openOrders', bot.openOrders()) # получили все ордера на продажу bbbot /2
        now_order=0 #  останется ложью если не будет ещё одного ордера по этой же валюте
        sum_of_bot=0
        mozno=True
#         binance_check+=5
        sells,bays = [],[]
        for i in orders[1]:
            if i['side']=='SELL'and i['clientOrderId'][0:5]=='botic':
                sells.append(i)
            elif i['side']=='BUY'and i['clientOrderId'][0:5]=='botic':
                bays.append(i)

        for sell in sells:
            if sell['symbol']==buys_symbol: 
                now_order=info(sell)# заранее нашли нужный ордер даже если он не будет нужен
            pred_mass=info(sell)
            #перебираем все ордера созданные ботом
            sum_of_bot+=pred_mass[3]
        b=('ticker/price', bot.tickerPrice(symbol=buys_symbol))# bbbot /2
#         binance_check+=1
       
        if now_order!= 0:
            now_cena=float(b[1]['price'])
            old_cena = float(now_order[1].split('_')[1])/10000
            if float(now_order[3])>fiftin_USDT*1.5:
                print(" зашли в новое условие для перезакупя ")
                if now_cena>old_cena*(1-persent_perezakup):
                    print(sell['symbol'],now_cena,old_cena*(1-persent_perezakup))
                    print( " и не дали этого сделать")
                    mozno=False
            if now_cena>old_cena*0.995:
                mozno=False
            print(" может ли идти дальше ", mozno)
            
        print(111111111111111111111111111111111111111)   
        print(ogranich,sum_of_bot,test_podstrax,ogranich-sum_of_bot-test_podstrax)   
        print(111111111111111111111111111111111111111)   
        if ogranich-sum_of_bot-test_podstrax>=fiftin_USDT+2 and mozno==True :

            
            
            add_USDT=0
            add_q=0
            add_price=0
            
            if now_order!= 0:# отмени прошлвй ордер
                print("Начал отменять ордер аааааааааааааааааааааааааааааааааааааааааааа",now_order[2])
                print('cancelOrder', bot.cancelOrder(
                orderId=now_order[2]  ,
                symbol=buys_symbol,
                ))
#                 binance_check+=1
                add_USDT=float(now_order[3])# соханили желаемую прибыль
                add_q=float(now_order[4])#и сколько валюты хотели продать
                add_price=float(now_order[5])
                print("кончил отменять ордер аааааааааааааааааааааааааааааааааааааааааааа")
            for i in bot.exchangeInfo()['symbols']: #bbbot 1
                if i['symbol']==buys_symbol:
                    arg_round_price = (f(i['filters'][0]['tickSize']))# на сколько округлять цену
                    arg_round = (f(i['filters'][2]['stepSize']))# на сколько округлять объём валюты
#             binance_check+=1
            b=('ticker/price', bot.tickerPrice(symbol=buys_symbol))#bbbot 1
            cena=float(b[1]['price'])# текущая цена валюты
            number_q_sym = Decimal(fiftin_USDT/float(cena))#поделив баксы на цену узнаем сколько валюты мы можем купить за желаемые баксы
#             binance_check+=1
            q_sym=(number_q_sym.quantize(Decimal("1."+str('0'*arg_round)), ROUND_HALF_EVEN))# 



            # print(q_sym,"сколько валюты купили на 50 баксов")
            aaa=bot.createOrder(
            symbol=buys_symbol,
            recvWindow=5000,
            side='buy',
            type='MARKET',
            newClientOrderId= 'botic'+str(time.time_ns()),
            quantity=float(q_sym)
            )
            print(aaa)
            time.sleep(2)
#             binance_check+=1
            
            q_kupili_with_cum=Decimal(str(aaa['origQty']))#*0.999 вычисляем сколько реально купили
            fiftin_USDT =float(cena*float(aaa['origQty']))
            print(fiftin_USDT,999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999999)
            number_q_kupili_with_cum = Decimal(q_kupili_with_cum)
            q_kupili_with_cum=(number_q_kupili_with_cum.quantize(Decimal("1."+str('0'*arg_round)), ROUND_DOWN))#округляем объём валюты
            #                 #### новая правка 07 04 2021
            back_cena = Decimal(fiftin_USDT/float(q_kupili_with_cum))
            back_cena = (back_cena.quantize(Decimal("1."+str('0'*arg_round_price)), ROUND_HALF_EVEN))
            #                 #### конец правки
            number_end_Q = Decimal(Decimal(q_kupili_with_cum)+Decimal(str(add_q)))
            end_Q=(number_end_Q.quantize(Decimal("1."+str('0'*arg_round)), ROUND_HALF_EVEN))# округлил полученный объём

            if add_USDT==0:
                number_end_cena = Decimal((fiftin_USDT/float(end_Q))*float(1+byatia(period,buys_symbol)) ) 
                if telega:
                    textik='https://api.telegram.org/bot950940715:AAFy0d1stHb45OhKoufE607oHm9zzCnwSrc/sendMessage?chat_id=-1001360890249&text='
                    rrr= requests.get(textik+ str(buys_symbol)+ " купил в первый раз и продаю по курсу " + str(number_end_cena) + "с наценочкой " + str(float(1+byatia(period,buys_symbol))) )
            else:
                fiftin_USDT=fiftin_USDT*float(1+byatia(period,buys_symbol))
                number_end_cena = Decimal( ((add_USDT+fiftin_USDT)/float(end_Q)) )
                if telega:
                    textik='https://api.telegram.org/bot950940715:AAFy0d1stHb45OhKoufE607oHm9zzCnwSrc/sendMessage?chat_id=-1001360890249&text='
                    rrr= requests.get(textik+ str(buys_symbol)+ " перезакупился и теперь продаю по курсу " + str(number_end_cena) )
            end_cena=(number_end_cena.quantize(Decimal("1."+str('0'*arg_round_price)), ROUND_HALF_EVEN))# округлил полученную цену

            aaaa=bot.createOrder(
            symbol=buys_symbol,
            recvWindow=5000,
            side='sell',
            type='LIMIT',
            timeInForce='GTC',
            newClientOrderId= 'botic'+str(time.time_ns())[8:]+"_"+str(round(back_cena*10000,0) ),
            quantity=float(end_Q),
            price=float(end_cena)
            )
#             binance_check+=1
            
def proverOCHKA_short(my_symbol,interval,L_rsi,L_mfi):
    klins=('klines', bot.klines(
    symbol=my_symbol,
    interval=interval,
    limit=100
    ))
    
    rsi_v,rsi_s = rsi_of_symbol(klins) 
    mfi_v,mfi_s,jopa = mfi_of_symbol(klins) 
    macd_v,macd_s,line1_v,line2_v = macd_of_symbol(klins) 

    
    
                            
    return((mfi_s =="4x рост и падение" or rsi_s == "4x рост и падение" or rsi_s =="Пробил верхнюю границу сверху вниз" or mfi_s == "Пробил верхнюю границу сверху вниз" or rsi_v>L_rsi or mfi_v>L_mfi) and macd_v>0 and line1_v>0 and line2_v>0)
            
def short():
    orders=('openOrders', bot.openOrders())#bbbot \2
#     binance_check+=5
    sells,bays = [],[]
    for i in orders[1]:
        if i['side']=='SELL'and i['clientOrderId'][0:5]=='botic':
            sells.append(i)
        elif i['side']=='BUY'and i['clientOrderId'][0:5]=='botic':
            bays.append(i)
    print("\n Сейчас мы в шортах \n")
    
    for sell in sells:
        order=info(sell)
        if proverOCHKA_short(order[0],"30m",83,94):
            
            if telega:
                textik='https://api.telegram.org/bot950940715:AAFy0d1stHb45OhKoufE607oHm9zzCnwSrc/sendMessage?chat_id=-1001360890249&text='
                rrr= requests.get(textik+ " вот тут бы продал по маркету (шорт) валютку "+ order[0] )
            #тут я типа удалили этот ордер
            
            
            b=('ticker/price', bot.tickerPrice(symbol=sell['symbol']))# bbbot 1
#             binance_check+=1
            now_cena=float(b[1]['price'])
            file = "Sell_table.xlsx"
            xl = pd.ExcelFile(file)
            Table = xl.parse(xl.sheet_names[0])
            new_row = {'валютка':order[0], 'объём проданной валюты':order[4], 'цена проданной валюты':now_cena, 'цена отменённого ордера':order[5]} 
            Table = Table.append(new_row, ignore_index=True)
            Table.to_excel("Sell_table.xlsx",index=False)
#     list_of_symbols= list(Table['валютка'])
#     for name in list_of_symbols:
#         row_of_my_symbol = (Table[Table["валютка"] == name])
#         b=('ticker/price', bot.tickerPrice(symbol=name))# bbbot 1
#         now_cena=float(b[1]['price'])
#         if float(row_of_my_symbol['цена проданной валюты'])*0.98<now_cena:
#             # тут я типа покупаю по маркету, проверяю не успел ли пока ждал ещё чего купить ( если успел то отменяю тот ордер)
#             # и выставляю новый ордер на продажу с учётом всего этого
def perebor(times,periods,list_of_s,sleep):
    now = datetime.datetime.now()
#     print("Важное время",now.isoformat(),binance_check)
#     binance_check=0
#     print("Важное время",now.isoformat(),binance_check)
    b=('ticker/price', bot.tickerPrice(symbol='BNBUSDT')) # bbbot orders 1
    cena_BNB = float(b[1]['price']) 
#     binance_check+=1
    b=('account', bot.account()) # bbbot orders 5
#     binance_check+=5
    coins=b[1]['balances']
    for coin in coins:
        if coin['asset'] == 'BNB':
            total_BNB=float(coin['free'])# 1 узнали сколько совободных бабок
            break
    if (total_BNB*cena_BNB)<10:
        aaa=bot.createOrder(
        symbol='BNBUSDT',
        recvWindow=5000,
        side='buy',
        type='MARKET',
        newClientOrderId= 'botic'+str(time.time_ns()),
        quantity=0.05
        )
#         binance_check+=1
    print(total_BNB*cena_BNB)
        
    
    
    a=('openOrders', bot.openOrders()) # bbbot orders/2
#     binance_check+=5

    
    

    sells,bays = [],[]
    for i in a[1]:
        if i['side']=='SELL':
            sells.append(i)
        else:
            bays.append(i)
    OLD = {}
    OLD_podstrax ={}
    for sell in sells:
        if sell['clientOrderId'][0:5]=='botic':#перебираем все ордера созданные ботом
            pred_mass=info(sell)
            OLD.update({pred_mass[0]:pred_mass[6]})
            if float(pred_mass[3])>order_cena*1.5: # тоесть уже один раз купили и 1 раз перекупили
                OLD_podstrax.update({pred_mass[0]:True })
            else:
                OLD_podstrax.update({pred_mass[0]:False })
#     print(OLD_podstrax)            
               
    
    
    list_of_mean_macd=[]
    list_of_mean_rsi=[]
    list_of_my_mfi=[]
    list_of_my_symbols=[]
    list_of_mean_macd_status=[]
    list_of_mean_rsi_status=[]
    list_of_mean_mfi_status=[]
    blue_line=[]
    red_line=[]
    kurs=[]
    kolvo_rost=0
    for my_symbol in list_of_s:
        klins_for_btc=('klines', bot.klines(
                    symbol='BTCUSDT',
                    interval='15m',
                    limit=100))# bbbot 1
#         binance_check+=1
        end,mmax,mmin,oopen=[],[],[],[]

        for klin in klins_for_btc[1]:
            end.append(float(klin[4]))
            mmax.append(float(klin[2]))
            mmin.append(float(klin[3]))
            oopen.append(float(klin[1]))
        df = pd.DataFrame((zip(oopen,end,mmax,mmin)),columns =['Open','Close', 'High','Low'])
        a=SuperTrend(df, 14, 3)
        test_SP = True
        for check in a:
            if check == "down":
                test_SP=False
                print("СУПЕР ТРЕНД В ЖОПЕ, ЖДЁМ МИНУТУ")
                kolvo_rost+=1
                if sshort:
                    short()
                time.sleep(60)
                break
        if not (kolvo_rost+1)%10:
                if sshort:
                    short()
                if test_SP: 
                    OLD,OLD_podstrax = prodavator(OLD,OLD_podstrax)
        if test_SP:
        
    #         if not (kolvo_rost+1)%5:
    # #             time.sleep(30)


            

            pokypat=0
    #         print(my_symbol)
    #         try:
            now = datetime.datetime.now()
            day_vol=('ticker/24hr', bot.ticker24hr(
            symbol=my_symbol))# bbbot 1
#             binance_check+=1
            day_vol=float(day_vol[1]["quoteVolume"])



            klins_for_btc=('klines', bot.klines(
                    symbol='BTCUSDT',
                    interval='1m',
                    limit=100))# bbbot 1

#             binance_check+=1
            if my_symbol[-3:] == "BTC":
                day_vol=day_vol*float(klins_for_btc[1][0][4])



            test_5_d=('klines', bot.klines(
                        symbol=my_symbol,
                        interval='1d',
                        limit=5))# bbbot 1
#             binance_check+=1



            mini=[]
            maxi=[]
            for klin in test_5_d[1]:
                mini.append(float(klin[3]))
                maxi.append(float(klin[2]))
            mmmeans=[]
            for i,j in zip(mini,maxi):
                mmmeans.append((1-float(i)/float(j))*100)
            number = sum(mmmeans) / len(mmmeans)


            Check_3_per=0
            if day_vol>5000000 and len(test_5_d[1])>=5 and number>9 : # в начале проверяем больше ли миллиона объём рынка




                responseAAA=0
                responseAA=0
                responseA=0

                list_of_responseA = {
                periods[0]: "не надо",
                periods[1]: "не надо",
                periods[2]: "не надо"
                            }

                list_of_responseAAA = {
                periods[0]: "не надо",
                periods[1]: "не надо",
                periods[2]: "не надо"
                            }
                list_of_responseAA = {
                periods[0]: "не надо",
                periods[1]: "не надо",
                periods[2]: "не надо"
                            }

                for period in periods:

                    meanss=[]
                    vols=[]

                    klins=('klines', bot.klines(
                        symbol=my_symbol,
                        interval=period,
                        limit=times
                    ))# bbbot 1
#                     binance_check+=1
                    for klin in klins[1]:
                        meanss.append(float(klin[4]))
                        vols.append(float(klin[5]))
    #                 print(my_symbol)





                    rsi_v,rsi_s = rsi_of_symbol(klins) 
                    mfi_v,mfi_s,jopa = mfi_of_symbol(klins) 
                    macd_v,macd_s,line1_v,line2_v = macd_of_symbol(klins) 

                    klins_for_btc=('klines', bot.klines(
                        symbol='BTCUSDT',
                        interval=period,
                        limit=5
                    ))# bbbot 1
#                     binance_check+=1
                    regresX=[]
                    regresY=[]
                    for i in range(5):
                            regresY.append(float(klins[1][95+i][4]))
                            regresX.append(float(klins_for_btc[1][i][4]))
                    VAL = np.vstack((regresX, regresY))
                    cor=str(np.corrcoef(VAL)[1][0])

                    kyrs_btc=0
                    start_kyrs_btc=klins_for_btc[1][0][4]
                    for j in range(5):
                        kyrs_btc+=float(klins_for_btc[1][j][4])

                    delta_btc=100*(1-float(start_kyrs_btc)/(kyrs_btc/5))
                    delta_symbol = str(100*(1-meanss[times-6]/np.mean(meanss[times-6:times-1])))
                    status_S_B_C_value=status_S_B_C(100*(1-meanss[times-6]/np.mean(meanss[times-6:times-1])),delta_btc,float(np.corrcoef(VAL)[1][0]))
                    if (rsi_v<30 or mfi_v<20) and macd_v<0 and line1_v<0 and line2_v<0:
                        if ((float(delta_btc)<0 and abs(float(delta_btc) ) < abs(float(delta_symbol) )) or float(delta_btc)>0) and float(delta_symbol)<-0.5:
                            Check_3_per+=1

                    if ((float(delta_btc)<0 and abs(float(delta_btc)) < abs(float(delta_symbol))  ) or float(delta_btc)>0) and float(delta_symbol)<-0.5:
                        if (rsi_s =="Пробил нижнюю границу снизу вверх" or mfi_s == "Пробил нижнюю границу снизу вверх") or rsi_v<15 or mfi_v<5 :
                            if macd_v <0: 
                                if line1_v <0:
                                    if line2_v <0:
                                        if telega:
                                            textik='https://api.telegram.org/bot950940715:AAFy0d1stHb45OhKoufE607oHm9zzCnwSrc/sendMessage?chat_id=-1001360890249&text='
                                            rrr= requests.get(textik+"rang AAA , "+ status_S_B_C_value+ " "+my_symbol+' за 5 раз по '+ period +' изменился на '+delta_symbol +" процентов. Пробил нижнюю границу снизу вверх.  биток за 5 " + period+ " изменился на " + str(delta_btc) +" процентов. корреляция с битком = "+cor )
                                        list_of_responseAAA[period]="надо"
                                        responseAAA=1





    #             if responseA:

    #                 textik='https://api.telegram.org/bot950940715:AAFy0d1stHb45OhKoufE607oHm9zzCnwSrc/sendMessage?chat_id=-1001360890249&text='
    #                 rrr= requests.get(textik+ "по " + periods[0] + " - " + list_of_responseA[periods[0]] + " по " + periods[1] + " - " + list_of_responseA[periods[1]] + " по " + periods[2] + " - " + list_of_responseA[periods[2]])
    #                 pokypat = 1
    #             if responseAA:

    #                 textik='https://api.telegram.org/bot950940715:AAFy0d1stHb45OhKoufE607oHm9zzCnwSrc/sendMessage?chat_id=-1001360890249&text='
    #                 rrr= requests.get(textik+ "по " + periods[0] + " - " + list_of_responseAA[periods[0]] + " по " + periods[1] + " - " + list_of_responseAA[periods[1]] + " по " + periods[2] + " - " + list_of_responseAA[periods[2]])
    #                 pokypat = 11
                if responseAAA:
                    if telega:
                        textik='https://api.telegram.org/bot950940715:AAFy0d1stHb45OhKoufE607oHm9zzCnwSrc/sendMessage?chat_id=-1001360890249&text='
                        rrr= requests.get(textik+ "по " + periods[0] + " - " + list_of_responseAAA[periods[0]] + " по " + periods[1] + " - " + list_of_responseAAA[periods[1]] + " по " + periods[2] + " - " + list_of_responseAAA[periods[2]])
    #                 if list_of_responseAAA[periods[0]] == "надо" and list_of_responseAAA[periods[1]] == "надо" or list_of_responseAAA[periods[2]] == "надо":
                    pokypat = 111


            if Check_3_per==3: 
                if telega:
                    textik='https://api.telegram.org/bot950940715:AAFy0d1stHb45OhKoufE607oHm9zzCnwSrc/sendMessage?chat_id=-1001360890249&text='
                    rrr= requests.get(textik+"rang__A, Валютка " + my_symbol + " по всем 3 периодам внизу" )
                pokypat = 1

            end=0       
            if pokypat == 111 and my_symbol[-4:]=="USDT":
                for per in range(3):
                    if list_of_responseAAA[periods[per]]=="надо":
                        pokypator(my_symbol,order_cena,limit_balance,periods[per],OLD_podstrax)
                        end=1
                        break

            if end!=1 and pokypat == 11 and my_symbol[-4:]=="USDT":
                for per in range(3):
                    if list_of_responseAA[periods[per]]=="надо":
                        pokypator(my_symbol,order_cena,limit_balance,periods[per],OLD_podstrax)
                        end=1
                        break

            if end!=1 and pokypat == 1 and my_symbol[-4:]=="USDT":
                pokypator(my_symbol,order_cena,limit_balance,"15m",OLD_podstrax)





            kolvo_rost+=1                                            
            print("уже ", kolvo_rost,my_symbol,now.isoformat())
        
    #                      
       
    
    time.sleep(sleep)
    if telega:
        textik='https://api.telegram.org/bot950940715:AAFy0d1stHb45OhKoufE607oHm9zzCnwSrc/sendMessage?chat_id=-1001360890249&text='
        rrr= requests.get(textik+"ещё жив" )
                                                    
    clear_output(wait=True)
    
def balance():
    b=('account', bot.account())
    coins=b[1]['balances']
    nash = bot.exchangeInfo()
    for coin in coins:
        if float(coin['free'])>0:
            if coin['asset']!='USDT':
                try:
                    a=('ticker/price', bot.tickerPrice(symbol=coin['asset']+'USDT'))
                    for i in nash['symbols']:
                        if i['symbol']==coin['asset']:
                                minimalka=(i['filters'][3])
                    if (float(a[1]['price'])*float(coin['free']))>=float(minimalka['minNotional']):
                            print("на кошельке сейчас есть в долларах столько ",float(a[1]['price'])*float(coin['free'])," этой валютки ",coin['asset'])
                except:

                    print("значит фиат")
def statistic(para):
    a=('myTrades', bot.myTrades(
        symbol=para,
    ))
    selled_usdt=[]
    gived_usdt=[]
    selled_coins=[]
    gived_coins=[]

    for i in a[1]:
        print(i)
        if i['isBuyer']==1:
#             print(i)
            gived_coins.append(float(i['qty'])-float(i['commission']))
            selled_usdt.append(float(i['quoteQty']))
        else:  
            selled_coins.append(float(i['qty']))
            gived_usdt.append(float(i['quoteQty'])-float(i['commission']))

    zarabotali=sum(gived_usdt)
    potratili=sum(selled_usdt)
    print("________________________\nРассмотрим валюту " + para)
    print(potratili,"потратили баксов")
    print(zarabotali,"заработали баксов")
    ostatok_coins=sum(gived_coins)-sum(selled_coins)
    print(ostatok_coins,"осталось монет")
    q=('ticker/price', bot.tickerPrice(symbol=para))
    print("а в баксах это",ostatok_coins*float(q[1]['price']))
    print("текущая прибыль ",zarabotali-potratili )
    print("а должны были",potratili*1.052 )
    print("Следовательно мы недополучили",(potratili*1.052-zarabotali) )
    print("тогда теоретически для того что бы выйти на должный уровень прибыли нам надо продать оставшиеся монетки за \n",(potratili*1.052-zarabotali)/ostatok_coins,"\n-----------------------------------------\n" )        
def info(para):
    
    b=('ticker/price', bot.tickerPrice(symbol=para['symbol']))
    print(para['symbol'], 100*(1-float(b[1]['price']) / float(para['price']) ))
#     binance_check+=1
    
    
    delta=100*(1-float(b[1]['price']) / float(para['price']))
    id_order=(para['clientOrderId']) # а вот это говно будем проверять на созданность ботом
#     print(id_order)
        
        
    q_symbols=Decimal(str(para['origQty']))-Decimal(str(para['executedQty']))
    q_symbols=float(q_symbols)
    USDT=float(para['price'])*q_symbols # вложенно бабок
    ID_orderok=para['orderId']#негенерированная часть которую будем прибавлять к 
    
    return(para['symbol'],id_order,ID_orderok,USDT,q_symbols,para['price'],delta)
    


a=(bot.exchangeInfo())
b=a['symbols']

list_of_symbols=[]
for bb in b:
#     or 'BTC' in bb['symbol'][-3:]
    if ('USDT' in bb['symbol'][-4:] ) and not 'UP' in bb['symbol'] and not 'DOWN' in bb['symbol']and not 'BEAR' in bb['symbol'] and not 'BULL' in bb['symbol'] and not 'USDC' in bb['symbol'] :
        if bb['symbol'][0:3] != 'USD':
            list_of_symbols.append(bb['symbol'])

list_of_symbols.remove('STRATUSDT')

list_of_symbols.remove('BCCUSDT')

list_of_symbols.remove('MCOUSDT')

list_of_symbols.remove('LENDUSDT')

list_of_symbols.remove('VENUSDT')

list_of_symbols.remove('DAIUSDT')

list_of_symbols.remove('HCUSDT')

list_of_symbols.remove('AUDUSDT')

list_of_symbols.remove('EURUSDT')

list_of_symbols.remove('BKRWUSDT')

list_of_symbols.remove('TUSDUSDT')

list_of_symbols.remove('SUSDUSDT')

list_of_symbols.remove('BUSDUSDT')

list_of_symbols.remove('GBPUSDT')

list_of_symbols.remove('PAXUSDT')

list_of_symbols.remove('DODOUSDT')

list_of_symbols.remove('PAXGUSDT')

list_of_symbols.remove('OMUSDT')

In [3]:
def main():
    for _ in range(50000):

        perebor(100,['15m','5m','1m'],list_of_symbols,60)

In [ ]:
if __name__ == "__main__":
    while True:
        try:
            main()
        except Exception as e:
            print(f"[-] Error: {e}")
            time.sleep(120)

231.0242102847
NULSUSDT 43.43935309973046
WANUSDT 50.5568814638027
ANTUSDT 14.10873928115307
СУПЕР ТРЕНД В ЖОПЕ, ЖДЁМ МИНУТУ
СУПЕР ТРЕНД В ЖОПЕ, ЖДЁМ МИНУТУ
СУПЕР ТРЕНД В ЖОПЕ, ЖДЁМ МИНУТУ
СУПЕР ТРЕНД В ЖОПЕ, ЖДЁМ МИНУТУ
СУПЕР ТРЕНД В ЖОПЕ, ЖДЁМ МИНУТУ
СУПЕР ТРЕНД В ЖОПЕ, ЖДЁМ МИНУТУ
СУПЕР ТРЕНД В ЖОПЕ, ЖДЁМ МИНУТУ
СУПЕР ТРЕНД В ЖОПЕ, ЖДЁМ МИНУТУ
СУПЕР ТРЕНД В ЖОПЕ, ЖДЁМ МИНУТУ
СУПЕР ТРЕНД В ЖОПЕ, ЖДЁМ МИНУТУ
СУПЕР ТРЕНД В ЖОПЕ, ЖДЁМ МИНУТУ
СУПЕР ТРЕНД В ЖОПЕ, ЖДЁМ МИНУТУ
СУПЕР ТРЕНД В ЖОПЕ, ЖДЁМ МИНУТУ
СУПЕР ТРЕНД В ЖОПЕ, ЖДЁМ МИНУТУ
СУПЕР ТРЕНД В ЖОПЕ, ЖДЁМ МИНУТУ
СУПЕР ТРЕНД В ЖОПЕ, ЖДЁМ МИНУТУ
СУПЕР ТРЕНД В ЖОПЕ, ЖДЁМ МИНУТУ
СУПЕР ТРЕНД В ЖОПЕ, ЖДЁМ МИНУТУ
СУПЕР ТРЕНД В ЖОПЕ, ЖДЁМ МИНУТУ
СУПЕР ТРЕНД В ЖОПЕ, ЖДЁМ МИНУТУ
СУПЕР ТРЕНД В ЖОПЕ, ЖДЁМ МИНУТУ
СУПЕР ТРЕНД В ЖОПЕ, ЖДЁМ МИНУТУ
СУПЕР ТРЕНД В ЖОПЕ, ЖДЁМ МИНУТУ
СУПЕР ТРЕНД В ЖОПЕ, ЖДЁМ МИНУТУ
СУПЕР ТРЕНД В ЖОПЕ, ЖДЁМ МИНУТУ
СУПЕР ТРЕНД В ЖОПЕ, ЖДЁМ МИНУТУ
СУПЕР ТРЕНД В ЖОПЕ, ЖДЁМ МИНУТУ
СУПЕР ТРЕНД В ЖОПЕ, ЖДЁМ МИНУТУ
СУПЕР ТРЕНД